## EMI-FastGRNN Implementation
This notebook contains the implementation of EMI-FastGRNN (one among the EMI algorithms). 
This notebbok consists of four parts:
1. The constructing of an entirely new graph with randomly initialized weights, that can be trained.
2. A saved model/graph can be loaded using *EMI-Driver*.
3. New graph created using the weights from a saved graph.
4. (Experimental)Restoration of weights and biases from numpy matrices

This notebook uses the features extracted from UCF_Crime dataset and is an extension of the notebook found in https://github.com/microsoft/EdgeML

In [2]:
from __future__ import print_function
import os
import sys
import tensorflow as tf
import numpy as np
# To include edgeml in python path
sys.path.insert(0, '../../')
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

# MI-RNN and EMI-RNN imports
from rnn import EMI_DataPipeline
from rnn import EMI_FastGRNN,EMI_FastRNN
from emirnnTrainer import EMI_Trainer, EMI_Driver
import utils

Let us set up some network parameters for the computation graph.

In [20]:
NUM_HIDDEN = 128
NUM_TIMESTEPS = 16
NUM_FEATS = 540
FORGET_BIAS = 1.0
NUM_OUTPUT = 8
USE_DROPOUT = 0

KEEP_PROB = 0.9
UPDATE_NL = "quantTanh"
GATE_NL = "quantSigm"
WRANK = 5
URANK = 6
PREFETCH_NUM = 5
BATCH_SIZE = 32
NUM_EPOCHS = 2
NUM_ITER = 4
NUM_ROUNDS = 10

# A staging direcory to store models
MODEL_PREFIX = '/home/adithyapa4444_gmail_com/CRIME_halfFPS_fastgrnn/model-fastgrnn'

In [4]:
def emirnn_preprocess(bagsize,no_of_features,extractedDir,numClass,subinstanceLen, subinstanceStride,data_csv,raw_create):
    """
    emirnn_preprocess(bagsize,no_of_features,extractedDir,numClass,subinstanceLen, subinstanceStride,data_csv,raw_create)
    
    extractedDir: location where the raw file and output models have to be stored
    
    data_csv: the extracted hof file.
    
    raw_create: 1 if  raw file already exists in directory 
               0 if  need to create file 
    
    """
    import pandas as pd
    import numpy as np
    import os 
    import video_stuff
    numSteps=bagsize
    numFeats=no_of_features
    try:
        dataset_name = pd.read_csv(data_csv,index_col=0)
    except:
        dataset_name = data_csv
    labels = dataset_name['label']


    labels=pd.DataFrame(labels)

    dataset_name.drop(['video_path','label'],axis=1,inplace=True)
    filtered_train = dataset_name
    filtered_target = labels

    
    print(filtered_train.shape)
    print(filtered_target.shape)


    y = filtered_target.values.reshape(int(len(filtered_target)/bagsize),bagsize)    #input bagsize

    x = filtered_train.values

                                                                                      #no_of_features=540
    x = x.reshape(int(len(x) / bagsize),bagsize, no_of_features)  


    print(x.shape)                         #(Bags, Timesteps, Features)



    one_hot_list = []
    for i in range(len(y)):
        one_hot_list.append(set(y[i]).pop())

    categorical_y_ver = one_hot_list
    categorical_y_ver = np.array(categorical_y_ver)


    def one_hot(y, numOutput):
        y = np.reshape(y, [-1])
        ret = np.zeros([y.shape[0], numOutput])
        for i, label in enumerate(y):
            ret[i, label] = 1
        return ret


    from sklearn.model_selection import train_test_split
    import pathlib


    x_train_val_combined, x_test, y_train_val_combined, y_test = train_test_split(x, categorical_y_ver, test_size=0.20, random_state=13)


                                                                                            #extractedDir = '/home/adithyapa4444_gmail_com/'

    timesteps = x_train_val_combined.shape[-2] #125
    feats = x_train_val_combined.shape[-1]  #9

    trainSize = int(x_train_val_combined.shape[0]*0.9) #6566
    x_train, x_val = x_train_val_combined[:trainSize], x_train_val_combined[trainSize:] 
    y_train, y_val = y_train_val_combined[:trainSize], y_train_val_combined[trainSize:]

    # normalization
    x_train = np.reshape(x_train, [-1, feats])
    mean = np.mean(x_train, axis=0)
    std = np.std(x_train, axis=0)

    # normalize train
    x_train = x_train - mean
    x_train = x_train / std
    x_train = np.reshape(x_train, [-1, timesteps, feats])

    # normalize val
    x_val = np.reshape(x_val, [-1, feats])
    x_val = x_val - mean
    x_val = x_val / std
    x_val = np.reshape(x_val, [-1, timesteps, feats])

    # normalize test
    x_test = np.reshape(x_test, [-1, feats])
    x_test = x_test - mean
    x_test = x_test / std
    x_test = np.reshape(x_test, [-1, timesteps, feats])

    # shuffle test, as this was remaining
    idx = np.arange(len(x_test))
    np.random.shuffle(idx)
    x_test = x_test[idx]
    y_test = y_test[idx]
    extractedDir += '/'
    if raw_create==0:
        if (os.path.isdir(extractedDir+'RAW')):
            print("A raw file already exsist in this directory")
            return
        else:
            numOutput = numClass
            y_train = one_hot(y_train, numOutput)
            y_val = one_hot(y_val, numOutput)
            y_test = one_hot(y_test, numOutput)
            

            pathlib.Path(extractedDir + 'RAW').mkdir(parents=True, exist_ok = True)

            np.save(extractedDir + "RAW/x_train", x_train)
            np.save(extractedDir + "RAW/y_train", y_train)
            np.save(extractedDir + "RAW/x_test", x_test)
            np.save(extractedDir + "RAW/y_test", y_test)
            np.save(extractedDir + "RAW/x_val", x_val)
            np.save(extractedDir + "RAW/y_val", y_val)
    numOutput = numClass
    y_train = one_hot(y_train, numOutput)
    y_val = one_hot(y_val, numOutput)
    y_test = one_hot(y_test, numOutput)

    data = {
        'x_train' : x_train, 
        'y_train' : y_train, 
        'x_val' : x_val, 
        'y_val' : y_val, 
        'x_test' : x_test, 
        'y_test' : y_test
    }
            
        

    def loadData(dirname):
        x_train = np.load(dirname + '/' + 'x_train.npy')
        y_train = np.load(dirname + '/' + 'y_train.npy')
        x_test = np.load(dirname + '/' + 'x_test.npy')
        y_test = np.load(dirname + '/' + 'y_test.npy')
        x_val = np.load(dirname + '/' + 'x_val.npy')
        y_val = np.load(dirname + '/' + 'y_val.npy')
        return x_train, y_train, x_test, y_test, x_val, y_val


    def bagData(X, Y, subinstanceLen, subinstanceStride,numClass,numSteps,numFeats):
        '''
        Takes x and y of shape
        [-1, 128, 9] and [-1, 6] respectively and converts it into bags of instances.
        returns [-1, numInstance, ]
        '''
        #numClass = 2
        #numSteps = 24 # Window length
        #numFeats = 540 # Number of features
        print("subinstanceLen:",subinstanceLen,"\nsubinstanceStride :", subinstanceStride,"\nnumClass:",numClass,"\nnumSteps",numSteps,"\nnumFeats",numFeats)
        print("X Shape :",X.shape)
        print("Y Shape :",Y.shape)
        assert X.ndim == 3
        assert X.shape[1] == numSteps
        assert X.shape[2] == numFeats
        assert subinstanceLen <= numSteps
        assert subinstanceLen > 0 # subinstance length = Number of readings for which the class signature occurs
        assert subinstanceStride <= numSteps  
        assert subinstanceStride >= 0 
        assert len(X) == len(Y)
        assert Y.ndim == 2
        assert Y.shape[1] == numClass
        x_bagged = []
        y_bagged = []
        for i, point in enumerate(X[:, :, :]):
            instanceList = []
            start = 0
            end = subinstanceLen
            while True:
                x = point[start:end, :]
                if len(x) < subinstanceLen:
                    x_ = np.zeros([subinstanceLen, x.shape[1]])
                    x_[:len(x), :] = x[:, :]
                    x = x_
                instanceList.append(x)
                if end >= numSteps:
                    break
                start += subinstanceStride
                end += subinstanceStride
            bag = np.array(instanceList)
            numSubinstance = bag.shape[0]
            label = Y[i]
            label = np.argmax(label)
            labelBag = np.zeros([numSubinstance, numClass])
            labelBag[:, label] = 1
            x_bagged.append(bag)
            label = np.array(labelBag)
            y_bagged.append(label)
        return np.array(x_bagged), np.array(y_bagged)

                                                                                                            #sourceDir, outDir

    def makeEMIData(subinstanceLen, subinstanceStride, data, sourceDir, outDir,numClass,numSteps,numFeats):
        x_train, y_train, x_test, y_test, x_val, y_val = loadData(sourceDir)
        try:
            x_train, y_train = bagData(x_train, y_train, subinstanceLen, subinstanceStride,numClass,numSteps,numFeats)
    #         np.save(outDir + '/x_train.npy', x)
    #         np.save(outDir + '/y_train.npy', y)
            print('Num train %d' % len(x))
            x_test, y_test = bagData(x_test, y_test, subinstanceLen, subinstanceStride,numClass,numSteps,numFeats)
    #         np.save(outDir + '/x_test.npy', x)
    #         np.save(outDir + '/y_test.npy', y)
            print('Num test %d' % len(x))
            x_val, y_val = bagData(x_val, y_val, subinstanceLen, subinstanceStride,numClass,numSteps,numFeats)
            print('Num val %d' % len(x))
        except:
            x_train, y_train = bagData(data['x_train'], data['y_train'], subinstanceLen, subinstanceStride,numClass,numSteps,numFeats)
    #         np.save(outDir + '/x_train.npy', x)
    #         np.save(outDir + '/y_train.npy', y)
            print('Num train %d' % len(x))
            x_test, y_test = bagData(data['x_test'], data['y_test'], subinstanceLen, subinstanceStride,numClass,numSteps,numFeats)
    #         np.save(outDir + '/x_test.npy', x)
    #         np.save(outDir + '/y_test.npy', y)
            print('Num test %d' % len(x))
            x_val, y_val = bagData(data['x_val'], data['y_val'], subinstanceLen, subinstanceStride,numClass,numSteps,numFeats)
            print('Num val %d' % len(x))
#         np.save(outDir + '/x_val.npy', x)
#         np.save(outDir + '/y_val.npy', y)
        
        return(x_train, y_train, x_val, y_val, x_test, y_test)



                                                                                        #subinstanceLen = 12
                                                                                        #subinstanceStride = 3
                                                                                        #extractedDir = '/home/adithyapa4444_gmail_com'
    rawDir = extractedDir + 'RAW'
    sourceDir = rawDir
    from os import mkdir
    # WHEN YOU CHANGE THE ABOVE - CREATE A FOLDER 
    if (not(os.path.isdir(extractedDir+'/'+str(subinstanceLen)+'_'+str(subinstanceStride)))):
        mkdir(extractedDir+'/'+str(subinstanceLen)+'_'+str(subinstanceStride))  

    outDir = extractedDir + '%d_%d' % (subinstanceLen, subinstanceStride)
    return(makeEMIData(subinstanceLen, subinstanceStride, data, sourceDir, outDir,numClass,numSteps,numFeats))


In [5]:
def get_hof(videopath,reduction_factor = 2):
    cap = cv2.VideoCapture(videopath)
    fps_counter = 0
    feat_sets = []
    ret, frame_prev = cap.read()
    if not ret:
        return
    frame_prev = cv2.cvtColor(frame_prev, cv2.COLOR_BGR2GRAY)
    frame_prev = cv2.resize(frame_prev, (120,160))
    while cap.isOpened():
        if (fps_counter%reduction_factor==0): #Reducing the fps by n (Reading every nth frame)
            ret, frame_next = cap.read()
            if not ret:
                break
            frame_next = cv2.cvtColor(frame_next, cv2.COLOR_BGR2GRAY)
            frame_next = cv2.resize(frame_next, (120,160))
            feat_sets.append(hof(flow = getFlow(frame_prev, frame_next), orientations = 5, cells_per_block = (6,6), pixels_per_cell = (20,20)))
            frame_prev = frame_next.copy()
        else:
            _,_ = cap.read()
            
            
            fps_counter+=1
    feat_sets = np.asarray(feat_sets)
    col_names = list('feat_' + str(i) for i in range(feat_sets.shape[-1]))
    #hog_feats = np.asarray(hog_feats)
    tuple_feats = list(map(lambda x: tuple(feat_sets[x]), range(len(feat_sets))))
    df = pd.DataFrame(tuple_feats, columns = col_names)
    df['video_path'] = videopath
    return df

In [6]:
import make_hof_half
from make_hof_half import get_processed_hof
import pandas as pd
import os,datetime
import matplotlib.pyplot as plt 
import numpy as np
import cv2
import pathlib
import pickle as pkl
from video_stuff import get_frames_per
from video_stuff import hof,getFlow
from time import time
from metrics import emi_model_metrics
import numpy as np
import matplotlib

In [22]:
# start = time()
hof_df = get_processed_hof(src_folder='/home/adithyapa4444_gmail_com/UCF_Crime/Full_UCrime/annotated/',bagsize=32,reduction_factor=2)
# time_string = "Bagsize : "+str(4)+"\nReduction Factor : "+str(6)+"\nTime :" + str(time()-start) + "\nNumber of bags:" + str(len(hof_df) // bag)
# file_1.write(time_string)
# file_1.close()
df_copy = hof_df
x_train, y_train, x_val, y_val, x_test, y_test = emirnn_preprocess(bagsize=12,
                  no_of_features=540,
                  extractedDir='/home/adithyapa4444_gmail_com/UCF_Crime/half/',
                  numClass=8,
                  subinstanceLen=16,
                  subinstanceStride=4,
                  data_csv=hof_df,
                  raw_create=1
                 )

data = {
    'x_train' : x_train, 
    'y_train' : y_train, 
    'x_val' : x_val, 
    'y_val' : y_val, 
    'x_test' : x_test, 
    'y_test' : y_test
}

/home/adithyapa4444_gmail_com/make_hof_half.py:87: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  dfa=pd.concat([dfa,df])


(495168, 540)
(495168, 1)
(41264, 12, 540)
subinstanceLen: 16 
subinstanceStride : 4 
numClass: 8 
numSteps 12 
numFeats 540
X Shape : (5273, 64, 540)
Y Shape : (5273, 8)
subinstanceLen: 16 
subinstanceStride : 4 
numClass: 8 
numSteps 12 
numFeats 540
X Shape : (29709, 12, 540)
Y Shape : (29709, 8)


AssertionError: 

# Loading Data

In [ ]:
path="/home/adithyapa4444_gmail_com/UCF_Crime/Full_UCrime_4d/64_16/"
# x_train, y_train = np.load(path + 'x_train.npy'), np.load(path + 'y_train.npy')
# x_test, y_test = np.load(path + 'x_test.npy'), np.load(path + 'y_test.npy')
# x_val, y_val = np.load(path + 'x_val.npy'), np.load(path + 'y_val.npy')

# BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
# step of EMI/MI RNN
BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
NUM_SUBINSTANCE = x_train.shape[1]
print("x_train shape is:", x_train.shape)
print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_val.shape)
print("y_test shape is:", y_val.shape)

In [10]:
x_test = x_test[:1]
y_test = y_test[:1]
np.save(path + 'x_test_pi',x_test)
np.save(path + 'y_test_pi',y_test)

In [ ]:
# Define the linear secondary classifier
def createExtendedGraph(self, baseOutput, *args, **kwargs):
    W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
    B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
    y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
    self.output = y_cap
    self.graphCreated = True
    
def addExtendedAssignOps(self, graph, W_val=None, B_val=None):
    W1 = graph.get_tensor_by_name('W1:0')
    B1 = graph.get_tensor_by_name('B1:0')
    W1_op = tf.assign(W1, W_val)
    B1_op = tf.assign(B1, B_val)
    self.assignOps.extend([W1_op, B1_op])

def restoreExtendedGraph(self, graph, *args, **kwargs):
    y_cap = graph.get_tensor_by_name('y_cap_tata:0')
    self.output = y_cap
    self.graphCreated = True
    
def feedDictFunc(self, keep_prob=None, inference=False, **kwargs):
    if inference is False:
        feedDict = {self._emiGraph.keep_prob: keep_prob}
    else:
        feedDict = {self._emiGraph.keep_prob: 1.0}
    return feedDict

EMI_FastGRNN._createExtendedGraph = createExtendedGraph
EMI_FastGRNN._restoreExtendedGraph = restoreExtendedGraph
EMI_FastGRNN.addExtendedAssignOps = addExtendedAssignOps

def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
    assert instanceOut.ndim == 2
    classes = np.argmax(instanceOut, axis=1)
    prob = np.max(instanceOut, axis=1)
    index = np.where(prob >= minProb)[0]
    if len(index) == 0:
        assert (len(instanceOut) - 1) == (len(classes) - 1)
        return classes[-1], len(instanceOut) - 1
    index = index[0]
    return classes[index], index


if USE_DROPOUT is True:
    EMI_Driver.feedDictFunc = feedDictFunc

## 1. Initializing a New Computation Graph

In the most common use cases, a new EMI-RNN graph would be created and trained

In [ ]:
tf.reset_default_graph()

inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
emiLSTM = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                              gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy')

# Construct the graph
g1 = tf.Graph()    
with g1.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)


Lets initialize a new session with this graph and train a model. The saved model will be used later for restoring.

In [ ]:
emiDriver.initializeSession(g1)
y_updated, modelStats = emiDriver.run(numClasses=NUM_OUTPUT, x_train=x_train,
                                     y_train=y_train, bag_train=BAG_TRAIN,
                                     x_val=x_val, y_val=y_val, bag_val=BAG_VAL,
                                     numIter=NUM_ITER, keep_prob=KEEP_PROB,
                                     numRounds=NUM_ROUNDS, batchSize=BATCH_SIZE,
                                     numEpochs=NUM_EPOCHS, modelPrefix=MODEL_PREFIX,
                                     fracEMI=0.5, updatePolicy='top-k', k=1)

As the output above indicates, the last restored model is `/tmp/model-lstm-1001`. That is, with `MODEL_PREFIX = '/tmp/model-lstm'` and `GLOBAL_STEP=1001`.

In [18]:
def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
    assert instanceOut.ndim == 2
    classes = np.argmax(instanceOut, axis=1)
    prob = np.max(instanceOut, axis=1)
    index = np.where(prob >= minProb)[0]
    if len(index) == 0:
        assert (len(instanceOut) - 1) == (len(classes) - 1)
        return classes[-1], len(instanceOut) - 1
    index = index[0]
    return classes[index], index

emiDriver.initializeSession(g1)

k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))

k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb, minProb=0.99)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
print('Additional savings: %f' % getEarlySaving(predictionStep, NUM_TIMESTEPS))

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("EMI/input-pipeline/inpX:0", shape=(?, 4, 3, 540), dtype=float32) is not an element of this graph.

## 2. Loading a Saved Graph into EMI-Driver

We will reset the computation graph, load a saved graph into the current `EMI_Driver` and verify its outputs.

In [8]:
MODEL_PREFIX

'/home/adithyapa4444_gmail_com/KTH_fastgrnn/model-fastgrnn'

In [18]:
tf.reset_default_graph()
emiDriver.initializeSession(g1)
emiDriver.loadSavedGraphToNewSession(MODEL_PREFIX , 1037)
k = 2

INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/CRIME_halfFPS_fastgrnn/model-fastgrnn-1037


In [17]:
%%time
k=3
import time
start = time.time()
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print(time.time() - start)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
def getEarlySaving(predictionStep, numTimeSteps, returnTotal=False):
    predictionStep = predictionStep + 1
    predictionStep = np.reshape(predictionStep, -1)
    totalSteps = np.sum(predictionStep)
    maxSteps = len(predictionStep) * numTimeSteps
    savings = 1.0 - (totalSteps / maxSteps)
    if returnTotal:
        return savings, totalSteps
    return savings
print('Additional savings: %f' % getEarlySaving(predictionStep, NUM_TIMESTEPS))

0.0051479339599609375
Accuracy at k = 3: 0.520548
Additional savings: 0.791667
CPU times: user 9.17 ms, sys: 39 µs, total: 9.21 ms
Wall time: 5.46 ms


In [19]:
x_test.shape

(1, 4, 6, 540)

In [28]:
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


AttributeError: 'bool' object has no attribute 'astype'

## 3. Initializing using a Saved Graph

Here we will construct a new computation graph, but will use a previously trained model to initialize it.

In [11]:
# Making sure the old graph and sessions are closed
sess = emiDriver.getCurrentSession()
sess.close()
tf.reset_default_graph()

Use the `GraphManager` to load the saved graph and load it into a new session.

In [13]:
tf.reset_default_graph()

sess = tf.Session()
graphManager = utils.GraphManager()
graph = graphManager.loadCheckpoint(sess, MODEL_PREFIX, globalStep=1037)

INFO:tensorflow:Restoring parameters from /home/adithyapa4444_gmail_com/CRIME_halfFPS_fastgrnn/model-fastgrnn-1037


Construct the forward graph as before, but provide the loaded `graph` as an argument to `__init__`.

In [14]:
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT, graph=graph)
emiLSTM = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                               gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy', graph=graph)

g1 = graph
with g1.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)

Let `EMI_Driver` know that we already have a session in place.

In [15]:
emiDriver.setSession(sess)

In [16]:
%%time

# tf.reset_default_graph()
# emiDriver.initializeSession(g1)
# emiDriver.loadSavedGraphToNewSession(MODEL_PREFIX, 1007)
k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                            minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))


Accuracy at k = 2: 0.848464
CPU times: user 3.51 s, sys: 1.54 s, total: 5.04 s
Wall time: 3.55 s


In [18]:
import time
from sklearn.metrics import classification_report, confusion_matrix
for _ in range(3):
    start = time.time()
    predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                                minProb=0.99, keep_prob=1.0)
    bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)

    end = time.time()
y_test_bag = np.argmax(y_test[:, 0, :], axis=1)
metrics_dict = {'time_run': end - start,
                'y_pred': bagPredictions,
                'y_test': y_test_bag,
                'classification_report': classification_report(bagPredictions, y_test_bag),
                'confusion_matrix': confusion_matrix(bagPredictions, y_test_bag)}


In [21]:
for metric in metrics_dict:
    print('\n')
    print(metric)
    print(metrics_dict[metric])
    print('----------------------------')



time_run
1.5736732482910156
----------------------------


y_pred
[5 5 1 ... 5 5 2]
----------------------------


y_test
[6 5 1 ... 5 5 2]
----------------------------


classification_report
              precision    recall  f1-score   support

           0       0.60      0.59      0.60        93
           1       0.73      0.81      0.77       172
           2       0.28      0.46      0.35        24
           3       0.36      0.33      0.35        12
           4       0.53      0.70      0.61        33
           5       0.97      0.91      0.94      1062
           6       0.47      0.66      0.55        32
           7       0.52      0.76      0.62        37

    accuracy                           0.85      1465
   macro avg       0.56      0.65      0.60      1465
weighted avg       0.87      0.85      0.86      1465

----------------------------


confusion_matrix
[[ 55   6   3   2   6   6   7   8]
 [  4 139   4   2   3  10   5   5]
 [  2   4  11   0   4   3   0   0]
 

In [22]:
1.5736732482910156 / 1465

0.0010741796916662223

In [10]:
x_test.shape, bagPredictions.shape,y_test.shape

((2947, 6, 48, 9), (2947,), (2947, 6, 6))

## 4. Restoring from Numpy Matrices

We first extract the model matrices from the graph and dump it into `.npy` files. Then we load it back again and initialize a new graph with these matrices.

In [27]:
graph = tf.get_default_graph()
W1 = graph.get_tensor_by_name('W1:0')
B1 = graph.get_tensor_by_name('B1:0')
allVars = emiLSTM.varList + [W1, B1]
sess = emiDriver.getCurrentSession()
allVars = sess.run(allVars)

base = '/tmp/models/'
np.save(base + 'kernel.npy', allVars[0])
np.save(base + 'bias.npy', allVars[1])
np.save(base + 'W1.npy', allVars[2])
np.save(base + 'B1.npy', allVars[3])

FailedPreconditionError: Attempting to use uninitialized value rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/B_g_1
	 [[Node: _retval_rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/B_g_1_0_2 = _Retval[T=DT_FLOAT, index=2, _device="/job:localhost/replica:0/task:0/device:CPU:0"](rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/B_g_1)]]

Reset the current session and graph

In [ ]:
sess = emiDriver.getCurrentSession()
sess.close()
tf.reset_default_graph()

Load the numpy matrices that will be used to initialize the graph.

In [28]:
base = '/home/iot/Documents/EdgeML-master/tf/examples/EMI-RNN/GRNN model'
kernel = np.load(base + 'kernel.npy')
bias = np.load(base + 'bias.npy')
W = np.load(base + 'W1.npy')
B = np.load(base + 'B1.npy')

Proceed with graph construction as normally done, except that we add the requisite assignment operations.

In [29]:
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS,
                                 NUM_OUTPUT)
emiLSTM = EMI_Fast(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS,
                        forgetBias=FORGET_BIAS, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy')

tf.reset_default_graph()
graph = tf.Graph()

with graph.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    # Add the assignment operations
    emiLSTM.addBaseAssignOps(graph, [kernel, bias])
    emiLSTM.addExtendedAssignOps(graph, W, B)
    # Setup the driver. You can run the initializations manually as well
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)

emiDriver.initializeSession(graph)
# Run the assignment operations
sess = emiDriver.getCurrentSession()
sess.run(emiLSTM.assignOps)

k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test,
                                                               earlyPolicy_minProb,
                                                               minProb=0.99,
                                                               keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k,
                                             numClass=NUM_OUTPUT)
print('PART IV: Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions ==
                                                        BAG_TEST).astype(int))))

NameError: name 'EMI_BasicLSTM' is not defined